In [ ]:
import os,sys,argparse;
from IPython.display import display,HTML,IFrame;
from plprofiler import plprofiler;
from plprofiler import plprofiler_tool;
from contextlib import closing;
parser = argparse.ArgumentParser();
args, _ = parser.parse_known_args();

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;


In [ ]:
profile_one = plprofiler();

cs = {
     "dbname"  : os.environ['POSTGRESQL_DB']
    ,"user"    : 'cipsrv'
    ,"password": os.environ['POSTGRESQL_CIP_PASS'] 
    ,"host"    : os.environ['POSTGRESQL_HOST']
    ,"port"    : os.environ['POSTGRESQL_PORT']
}

try:
    profile_one.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
str_sql = """
    DO $$DECLARE
        rec RECORD;
    BEGIN
        rec := cipsrv_engine.cipsrv_index(
            p_points                         := JSONB_BUILD_OBJECT(
                 'type'       ,'Point'
                ,'coordinates',JSONB_BUILD_ARRAY(-96.728783,46.88155)
            )
           ,p_lines                          := NULL
           ,p_areas                          := NULL
           ,p_geometry                       := NULL
           ,p_geometry_clip                  := NULL
           ,p_geometry_clip_stage            := NULL
           ,p_catchment_filter               := ARRAY['MN']
           ,p_nhdplus_version                := 'MR'
           ,p_wbd_version                    := NULL  
           ,p_default_point_indexing_method  := 'point_simple'
           ,p_default_line_indexing_method   := 'line_simple'
           ,p_default_line_threshold         := 10
           ,p_default_ring_indexing_method   := 'area_simple'
           ,p_default_ring_areacat_threshold := 10
           ,p_default_ring_areaevt_threshold := 50
           ,p_default_area_indexing_method   := 'area_simple'
           ,p_default_areacat_threshold      := 10
           ,p_default_areaevt_threshold      := 50
           ,p_known_region                   := '5070'
           ,p_return_indexed_features        := FALSE
           ,p_return_indexed_collection      := FALSE
           ,p_return_catchment_geometry      := TRUE
           ,p_return_indexing_summary        := TRUE
        );

    END;$$
""";

In [ ]:
profile_one.enable();
profile_one.reset_local();
profile_one.execute_sql(str_sql,sys.stdout);


In [ ]:
opt_top = 10;

config = {
     'name':         'profile one'
    ,'title':        'profile one'
    ,'tabstop':      3
    ,'svg_width':    1200
    ,'table_width':  '80%'
    ,'desc':         'profile one'
}
opt_name = config['name'];
report_data = profile_one.get_local_report_data(opt_name,opt_top,None);
report_data['config'] = config;

with open('./tmp_profile_one.html','w') as output_fd:
    profile_one.report(report_data,output_fd);


In [ ]:
display(IFrame(src='./tmp_profile_one.html',width=1080,height=1200));

In [ ]:
os.remove('./tmp_profile_one.html');
